# **Regressão Logística — Módulo 3, Notebook 3/6**

---

## Índice

1. [Introdução à Regressão Logística](#introducao)
2. [Como a Regressão Logística Funciona?](#como-funciona)
3. [Intuição Prática: Sobreviventes do Titanic](#intuicao)
4. [Formalização Matemática](#formalizacao)
5. [Implementação com Scikit-Learn](#sklearn)
6. [Técnicas Avançadas](#tecnicas)
7. [Vantagens e Desvantagens](#vantagens-desvantagens)

---

In [ ]:
# Importando as bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import kagglehub
from kagglehub import KaggleDatasetAdapter

<a id='introducao'></a>
## **Introdução à Regressão Logística**

Imagine que você trabalha em um hospital e precisa prever se um paciente terá complicações cardíacas com base em idade, pressão arterial e colesterol. Ou talvez você queira prever se um email é spam baseado nas palavras que contém. Esses são problemas de **classificação binária** — a resposta é "sim" ou "não", 0 ou 1.

A **Regressão Logística** é um dos algoritmos mais populares para resolver esse tipo de problema. Apesar do nome "regressão", ela é usada para **classificação** e nos fornece a **probabilidade** de uma amostra pertencer a uma determinada classe.

Diferentemente da regressão linear que prevê valores contínuos (como o preço de uma casa), a Regressão Logística prevê a probabilidade de uma entrada pertencer a uma classe específica. Essa probabilidade é sempre um valor entre 0 e 1, obtido através da **função sigmoide**:

$$\sigma(z) = \frac{1}{1 + e^{-z}}$$

A saída do modelo é uma probabilidade, e a classificação final é feita com um limiar (threshold), geralmente 0.5:

- Se $P(Y=1) \geq 0.5$, a classe prevista será 1
- Se $P(Y=1) < 0.5$, a classe prevista será 0

<a id='como-funciona'></a>
## **Como a Regressão Logística Funciona?**

A Regressão Logística funciona em três etapas fundamentais:

### **Etapa 1: Combinação Linear (Score)**

Primeiro, o algoritmo calcula um "score" combinando as features da amostra com pesos aprendidos:

$$z = w_1x_1 + w_2x_2 + ... + w_nx_n + b = \mathbf{w}^T\mathbf{x} + b$$

Este valor $z$ é chamado de **log-odds** (logaritmo da razão de chances) e pode variar de $-\infty$ a $+\infty$.

### **Etapa 2: Transformação pela Função Sigmoide**

O score $z$ não é intuitivo. Para obter uma probabilidade (valor entre 0 e 1), aplicamos a **função sigmoide**:

$$P(Y=1|X) = \sigma(z) = \frac{1}{1 + e^{-z}}$$

Propriedades importantes da sigmoide:
- Quando $z$ é muito positivo → $\sigma(z) \approx 1$ (alta probabilidade da classe 1)
- Quando $z$ é muito negativo → $\sigma(z) \approx 0$ (baixa probabilidade da classe 1)
- Quando $z = 0$ → $\sigma(z) = 0.5$ (probabilidade neutra)

### **Etapa 3: Classificação via Threshold**

Com a probabilidade em mãos, aplicamos um limiar de decisão (geralmente 0.5):

$$\hat{y} = \begin{cases} 
1, & \text{se } P(Y=1|X) \geq 0.5 \\
0, & \text{se } P(Y=1|X) < 0.5
\end{cases}$$

### **Log-Odds: A Intuição Matemática**

Podemos reescrever a equação de regressão logística em termos da razão de chances (odds ratio):

$$\log\left(\frac{P(Y=1)}{1 - P(Y=1)}\right) = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + ... + \beta_n X_n$$

Onde:
- $P(Y=1)$ é a probabilidade do evento ocorrer
- $\frac{P(Y=1)}{1 - P(Y=1)}$ representa a razão de chances (odds ratio)
- $\log(\frac{P(Y=1)}{1 - P(Y=1)})$ é o log-odds, que se relaciona linearmente com as features

**Interpretação dos coeficientes:**

Se $\beta_1 = 0.3$, para cada aumento de 1 unidade em $X_1$, o log-odds aumenta em 0.3. Para interpretar diretamente o efeito na razão de chances, aplicamos a exponenciação:

$$e^{\beta_1} = e^{0.3} \approx 1.35$$

Isso significa que as chances do evento positivo aumentam em aproximadamente 35% para cada unidade adicional de $X_1$.

<a id='intuicao'></a>
## **Intuição Prática: Sobreviventes do Titanic**

Vamos consolidar esse conhecimento através de um exemplo prático e histórico: o naufrágio do Titanic em 1912.

### **O Problema**

Imagine que você tem informações sobre um passageiro:

- **Nome:** Jack
- **Idade:** 22 anos
- **Classe:** 3ª classe
- **Sexo:** Masculino

**Pergunta:** Será que conseguimos prever se Jack sobreviveu ao naufrágio baseado apenas nessas características?

Para responder isso, vamos:

1. Analisar os dados históricos dos passageiros do Titanic
2. Identificar padrões de sobrevivência
3. Treinar um modelo de Regressão Logística
4. Fazer a predição para Jack

Vamos construir o raciocínio do algoritmo passo a passo.

In [ ]:
# Carregando o dataset do Titanic
df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "yasserh/titanic-dataset",
    "Titanic-Dataset.csv",
    pandas_kwargs={"encoding": "latin", "usecols": [1, 2, 4, 5]}
)

print(f"Total de passageiros: {len(df)}")
print(f"\nPrimeiras 5 linhas:")
df.head()

In [ ]:
# Análise exploratória inicial
print(f"Dataset shape: {df.shape}")
print("\nColunas disponíveis:", df.columns.tolist())
print("\nInformações do dataset:")
df.info()

print("\nEstatísticas descritivas:")
print(df.describe())

### **Ponderando as Evidências**

Antes de treinar o modelo, vamos entender quais características influenciam a sobrevivência. A Regressão Logística aprenderá um "peso" para cada feature, indicando sua importância e direção:

- **Sexo = Feminino:** Será que ser mulher aumentava as chances de sobrevivência? ("Mulheres e crianças primeiro")
- **Classe = 1ª:** Passageiros da primeira classe tinham acesso prioritário aos botes salva-vidas?
- **Classe = 3ª:** Estar na terceira classe diminuiria as chances de sobreviver?
- **Idade:** Crianças e jovens teriam prioridade?

Vamos analisar os dados para descobrir!

In [ ]:
# Análise de sobrevivência por categoria

print("TAXA DE SOBREVIVÊNCIA POR SEXO:")
survival_by_sex = df.groupby("Sex")["Survived"].agg(["mean", "count"])
survival_by_sex["mean"] *= 100
print(survival_by_sex)

print("\n" + "="*60)
print("\nTAXA DE SOBREVIVÊNCIA POR CLASSE:")
survival_by_class = df.groupby("Pclass")["Survived"].agg(["mean", "count"])
survival_by_class["mean"] *= 100
print(survival_by_class)

print("\n" + "="*60)
print("\nTAXA DE SOBREVIVÊNCIA POR FAIXA ETÁRIA:")
df['Age_group'] = pd.cut(df['Age'], bins=[0, 16, 35, 60, 100], 
                         labels=['Child(0-16)', 'Young Adult(17-35)', 'Adult(36-60)', 'Senior(61+)'])

survival_by_age = df.groupby("Age_group", observed=True)["Survived"].agg(["mean", "count"])
survival_by_age["mean"] *= 100
print(survival_by_age)

In [ ]:
# Visualizações das taxas de sobrevivência

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Análise de Sobrevivência — Titanic Dataset', fontsize=16, fontweight='bold')

# Gráfico 1: Sobrevivência por Gênero
ax1 = axes[0, 0]
survival_sex = df.groupby('Sex')['Survived'].mean() * 100
bars = ax1.bar(survival_sex.index, survival_sex.values, color=['lightcoral', 'skyblue'], 
               alpha=0.7, edgecolor='black', linewidth=2)
ax1.set_ylabel('Taxa de Sobrevivência (%)')
ax1.set_ylim(0, 100)
ax1.set_title('Taxa de Sobrevivência por Gênero', fontsize=14)
ax1.grid(axis='y', linestyle='--', alpha=0.7)
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width() / 2, height + 1, 
             f'{height:.1f}%', ha='center', va='bottom')

# Gráfico 2: Sobrevivência por Classe
ax2 = axes[0, 1]
survival_class = df.groupby('Pclass')['Survived'].mean() * 100
bars = ax2.bar(survival_class.index.astype(str), survival_class.values, 
               color=['lightgreen', 'orange', 'lightgrey'], alpha=0.7, 
               edgecolor='black', linewidth=2)
ax2.set_ylabel('Taxa de Sobrevivência (%)')
ax2.set_ylim(0, 100)
ax2.set_title('Taxa de Sobrevivência por Classe do Passageiro', fontsize=14)
ax2.grid(axis='y', linestyle='--', alpha=0.7)
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width() / 2, height + 1, 
             f'{height:.1f}%', ha='center', va='bottom')

# Gráfico 3: Distribuição de Idade por Status de Sobrevivência
ax3 = axes[1, 0]
survived = df[df['Survived'] == 1]['Age']
died = df[df['Survived'] == 0]['Age']
ax3.hist([died, survived], bins=20, stacked=True, 
         color=['lightcoral', 'lightblue'], edgecolor='black', alpha=0.7)
ax3.set_xlabel('Idade')
ax3.set_ylabel('Número de Passageiros')
ax3.set_title('Distribuição de Idade por Status de Sobrevivência', fontsize=14)
ax3.legend(['Não Sobreviveu', 'Sobreviveu'])
ax3.grid(axis='y', linestyle='--', alpha=0.7)

# Gráfico 4: Heatmap Classe vs Gênero
ax4 = axes[1, 1]
pivot = df.pivot_table(values='Survived', index='Pclass', columns='Sex', aggfunc='mean')
sns.heatmap(pivot * 100, annot=True, fmt=".1f", cmap='YlGnBu', 
            cbar_kws={'label': 'Taxa de Sobrevivência (%)'}, ax=ax4, 
            linewidths=.5, linecolor='black')
ax4.set_title('Taxa de Sobrevivência por Classe e Gênero', fontsize=14)
ax4.set_ylabel('Classe do Passageiro')
ax4.set_xlabel('Gênero')

plt.tight_layout()
plt.show()

### **Insights dos Dados**

Os gráficos revelam padrões claros que a Regressão Logística irá aprender:

**1. Gênero foi crucial:**
- Mulheres: **74.2%** de sobrevivência
- Homens: **18.9%** de sobrevivência
- O protocolo "mulheres e crianças primeiro" foi real!

**2. Classe social importava:**
- 1ª classe: **63.0%** de sobrevivência
- 2ª classe: **47.3%** de sobrevivência
- 3ª classe: **24.2%** de sobrevivência
- Acesso aos botes salva-vidas era desigual

**3. Idade tinha influência:**
- Crianças (0-16 anos): **55%** de sobrevivência
- Adultos jovens (17-35 anos): **37%** de sobrevivência
- Crianças receberam prioridade

Esses padrões serão transformados em **pesos** pelo algoritmo de Regressão Logística!

### **Entendendo a Função Sigmoide Visualmente**

Antes de treinar o modelo, vamos visualizar como a função sigmoide transforma os scores (log-odds) em probabilidades:

In [ ]:
# Visualização da função sigmoide

def sigmoid(z):
    """Função sigmoide: transforma qualquer valor real em probabilidade (0-1)"""
    return 1 / (1 + np.exp(-z))

z_values = np.linspace(-10, 10, 400)
sigmoid_values = sigmoid(z_values)

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(z_values, sigmoid_values, 'b-', linewidth=2, label='Função Sigmóide')
ax.axhline(0.5, color='black', linestyle='--', linewidth=2, alpha=0.7, 
           label='Limiar de decisão (0.5)')
ax.axvline(0, color='black', linewidth=1, linestyle='--', alpha=0.5)
ax.fill_between(z_values, 0, sigmoid_values, where=(sigmoid_values >= 0.5), 
                color='lightblue', alpha=0.5, label='Classe 1 (Sobreviveu)')
ax.fill_between(z_values, 0, sigmoid_values, where=(sigmoid_values < 0.5), 
                color='lightcoral', alpha=0.5, label='Classe 0 (Não Sobreviveu)')

# Exemplos de pontos específicos
examples = [
    (-6, sigmoid(-6), "z=-6\nScore muito negativo"), 
    (-3, sigmoid(-3), "z=-3"), 
    (0, sigmoid(0), "z=0\nPonto neutro"), 
    (3, sigmoid(3), "z=3"), 
    (6, sigmoid(6), "z=6\nScore muito positivo")
]

for z, s, label in examples:  
    ax.plot(z, s, 'ro', markersize=10)
    ax.annotate(f'{label}\nP = {s:.3f}', xy=(z, s), xytext=(z, s + 0.05),
                arrowprops=dict(facecolor='wheat', arrowstyle='->'),
                fontsize=9, ha='center')

ax.set_title('Função Sigmóide: De Score para Probabilidade', fontsize=16, fontweight='bold')
ax.set_xlabel('Score (z) = w₁x₁ + w₂x₂ + ... + b', fontsize=14)
ax.set_ylabel('Probabilidade σ(z)', fontsize=14)
ax.set_ylim(-0.1, 1.1)
ax.legend()
ax.grid(linestyle='--', alpha=0.7)
plt.show()

**Interpretação:**

- **z muito negativo (ex: -6):** Score baixo → Probabilidade ~0% → Classe 0
- **z = 0:** Score neutro → Probabilidade 50% → Ponto de decisão
- **z muito positivo (ex: +6):** Score alto → Probabilidade ~100% → Classe 1

Agora que entendemos a sigmoide, vamos ver a formalização matemática completa!

<a id='formalizacao'></a>
## **Formalização Matemática**

Agora que você entendeu intuitivamente como a Regressão Logística funciona, vamos formalizar o algoritmo matematicamente.

**Log-Odds (Logit)**

A odd de um evento é a razão entre a probabilidade de ele ocorrer $(p)$ e a probabilidade de não ocorrer $(1-p)$

$ Odds = \frac{p}{1-p}$

A Regressão Logística assume que o logaritmo da odd (**log-odds ou logit**) é uma função linear das features:

$\ln \left(\frac{p}{1-p}\right) = \mathbf{w}^T\mathbf{x}+b$

Onde $\mathbf{w}$ e $\mathbf{x}$ são os vetores de pesos e features respectivamente.

Note que, ao isolar $p$ na equação, obtemos exatamente a função sigmoide.

### **Entrada**

**Conjunto de treinamento:** $D = \{(\mathbf{x}^{(1)}, y^{(1)}), (\mathbf{x}^{(2)}, y^{(2)}), ..., (\mathbf{x}^{(n)}, y^{(n)})\}$

Onde:
- $\mathbf{x}^{(i)} = (x^{(i)}_1, x^{(i)}_2, ..., x^{(i)}_d)$: vetor de $d$ features da amostra $i$
- Adicionamos o termo de intercepto $x_0^{(i)} = 1$, portanto $\mathbf{x}^{(i)} \in \mathbb{R}^{d+1}$
- $y^{(i)} \in \{0, 1\}$: rótulo de classe binário (0 ou 1)

### **Processamento**

O processamento consiste em aprender um vetor de parâmetros (pesos) $\mathbf{w} \in \mathbb{R}^{d+1}$ que melhor mapeia as entradas $\mathbf{x}$ para saídas $y$.

#### **1. Hipótese do Modelo**

A função hipótese $h_{\mathbf{w}, b}(\mathbf{x})$ retorna a probabilidade de $y=1$ dada a amostra $\mathbf{x}$:

$$h_{\mathbf{w}, b}(\mathbf{x}) = \sigma(\mathbf{w}^T\mathbf{x} + b) = \frac{1}{1 + e^{-(\mathbf{w}^T\mathbf{x} + b)}}$$

Onde:
- $\mathbf{w}$: vetor de pesos
- $b$: bias (intercepto)
- $\sigma$: função sigmoide

#### **2. Função de Custo (Log-Loss)**

Para uma única amostra de treinamento $(\mathbf{x}, y)$, a função custo é definida como:

$$L(\hat{p}, y) = -[y\log(\hat{p}) + (1-y)\log(1-\hat{p})]$$

Essa função penaliza predições erradas:
- Se a classe real é $y=1$, a perda é $-\log(\hat{p})$. Se o modelo prevê $\hat{p} \to 0$, a perda tende a $\infty$
- Se a classe real é $y=0$, a perda é $-\log(1-\hat{p})$. Se o modelo prevê $\hat{p} \to 1$, a perda tende a $\infty$

A função custo total $J(\mathbf{w}, b)$ para o conjunto de dados inteiro é a média das perdas individuais:

$$J(\mathbf{w}, b) = -\frac{1}{n}\sum_{i=1}^{n}[y^{(i)}\log(h_{\mathbf{w}, b}(\mathbf{x}^{(i)})) + (1-y^{(i)})\log(1-h_{\mathbf{w}, b}(\mathbf{x}^{(i)}))]$$

#### **3. Otimização via Gradiente Descendente**

Para minimizar $J(\mathbf{w}, b)$, utilizamos o algoritmo **Gradiente Descendente**. Ele atualiza iterativamente os parâmetros na direção oposta ao gradiente da função de custo.

As derivadas parciais da função de custo em relação a cada peso $w_j$ e ao bias $b$ são:

$$\frac{\partial J}{\partial w_j} = \frac{1}{n}\sum_{i=1}^{n}(h_{\mathbf{w}, b}(\mathbf{x}^{(i)}) - y^{(i)})x_j^{(i)}$$

$$\frac{\partial J}{\partial b} = \frac{1}{n}\sum_{i=1}^{n}(h_{\mathbf{w}, b}(\mathbf{x}^{(i)}) - y^{(i)})$$

As regras de atualização a cada iteração são:

$$w_j := w_j - \alpha\frac{\partial J}{\partial w_j}$$

$$b := b - \alpha\frac{\partial J}{\partial b}$$

Onde $\alpha$ é a **taxa de aprendizado** (learning rate).

### **Saída — Classificação**

**Parâmetros do Modelo:** Um vetor de pesos $\mathbf{w}$ e um bias $b$ otimizados.

**Predição:** Para uma nova amostra $\mathbf{x}^o$, o modelo calcula a probabilidade:

$$\hat{p} = \sigma(\mathbf{w}^T\mathbf{x}^o + b)$$

**Classificação:** Um limiar de decisão (threshold), geralmente 0.5, é aplicado à probabilidade para obter a classe final:

$$\hat{y} = \begin{cases} 
1, & \text{se } \hat{p} \geq 0.5 \\
0, & \text{se } \hat{p} < 0.5
\end{cases}$$

### **Considerações Importantes**

#### **1. Interpretação dos Coeficientes**

Os pesos ($w_j$) podem ser interpretados de forma intuitiva. Quando exponenciados, obtemos a razão de chances (odds ratio):

- Para cada aumento de uma unidade em $x_j$, as odds do resultado ser $y=1$ são multiplicadas por $e^{w_j}$

**Exemplo:** Se $w_{\text{idade}} = -0.05$, então $e^{-0.05} \approx 0.95$. Isso significa que para cada ano adicional de idade, as odds de sobreviver diminuem em 5%.

#### **2. Fronteira de Decisão (Decision Boundary)**

A fronteira de decisão é a superfície que separa as classes. No caso da Regressão Logística, ela ocorre onde a probabilidade é exatamente 0.5, o que corresponde a:

$$\mathbf{w}^T\mathbf{x} + b = 0$$

Note que a fronteira de decisão da Regressão Logística é **linear**.

#### **3. Regularização (L1 e L2)**

Para evitar overfitting, especialmente com um grande número de features, podemos adicionar termos de regularização:

- **Ridge (L2):** Adiciona $\frac{\lambda}{2n} \sum_{j=1}^{d}w_j^2$ para penalizar pesos grandes
- **Lasso (L1):** Adiciona $\frac{\lambda}{n} \sum_{j=1}^{d}|w_j|$ para forçar pesos a se tornarem exatamente zero

#### **4. Extensão para Classificação Multiclasse**

A Regressão Logística pode ser generalizada para problemas com mais de duas classes através da **Regressão Softmax**. Ela usa a função Softmax, em vez da sigmoide, para calcular a probabilidade de cada uma das $K$ classes:

$$P(y=k|\mathbf{x}) = \frac{e^{z_k}}{\sum_{j=1}^{K}e^{z_j}}$$

Onde $z_k = \mathbf{w}_k^T\mathbf{x} + b_k$ é o score para a classe $k$.

<a id='sklearn'></a>
## **Implementação com Scikit-Learn**

Na prática, usamos bibliotecas otimizadas como o Scikit-Learn. Vamos implementar uma solução completa para o problema do Titanic e **finalmente responder se Jack sobreviveu!**

### **Preparação dos Dados**

Vamos preparar o dataset do Titanic para treinar nosso modelo de Regressão Logística:

In [ ]:
# Preparação dos dados
# Remover linhas com valores faltantes em Age
df_clean = df.dropna(subset=['Age'])

# Codificar a coluna Sex (male=1, female=0)
df_clean['Sex'] = (df_clean['Sex'] == 'male').astype(int)

# Selecionar features (X) e target (y)
X = df_clean[['Pclass', 'Sex', 'Age']].values
y = df_clean['Survived'].values

print("Features selecionadas:")
print(f"  - Pclass (1ª, 2ª ou 3ª classe)")
print(f"  - Sex (0=female, 1=male)")
print(f"  - Age (idade em anos)")

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nConjunto de treino: {len(X_train)} amostras")
print(f"Conjunto de teste: {len(X_test)} amostras")

### **Criação e Treinamento do Modelo**

Agora vamos treinar o modelo de Regressão Logística:

In [ ]:
# Criar e treinar o modelo
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Fazer predições
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)

print("="*60)
print("RESULTADOS DO MODELO")
print("="*60)
print(f"\nAcurácia: {accuracy*100:.2f}%")
print(f"\nCoeficientes aprendidos (pesos):")
print(f"  w_Pclass (Classe): {model.coef_[0][0]:.4f}")
print(f"  w_Sex (Sexo): {model.coef_[0][1]:.4f}")
print(f"  w_Age (Idade): {model.coef_[0][2]:.4f}")
print(f"  b (bias/intercepto): {model.intercept_[0]:.4f}")

print("\n" + "="*60)
print("MATRIZ DE CONFUSÃO")
print("="*60)
cm = confusion_matrix(y_test, y_pred)
print(cm)

print("\n" + "="*60)
print("RELATÓRIO DE CLASSIFICAÇÃO")
print("="*60)
print(classification_report(y_test, y_pred, target_names=['Não Sobreviveu', 'Sobreviveu']))

### **Interpretação dos Pesos**

Vamos interpretar o que o modelo aprendeu:

In [ ]:
# Interpretação dos coeficientes via odds ratio

print("INTERPRETAÇÃO DOS PESOS (ODDS RATIO):")
print("="*60)

for i, feature in enumerate(['Pclass', 'Sex', 'Age']):
    coef = model.coef_[0][i]
    odds_ratio = np.exp(coef)
    
    print(f"\n{feature}:")
    print(f"  Coeficiente (w): {coef:.4f}")
    print(f"  Odds Ratio (e^w): {odds_ratio:.4f}")
    
    if odds_ratio > 1:
        print(f"  → Para cada aumento de 1 unidade, as odds de sobreviver aumentam {(odds_ratio-1)*100:.1f}%")
    else:
        print(f"  → Para cada aumento de 1 unidade, as odds de sobreviver diminuem {(1-odds_ratio)*100:.1f}%")

<a id='tecnicas'></a>
## **Técnicas Avançadas**

Vamos explorar técnicas mais sofisticadas para melhorar e analisar o modelo de Regressão Logística.

### **Técnica 1: Ajuste do Threshold**

O limiar de decisão padrão é 0.5, mas podemos ajustá-lo dependendo do contexto do problema.

In [ ]:
# Testando diferentes thresholds

thresholds = [0.3, 0.5, 0.7]

print("IMPACTO DO AJUSTE DE THRESHOLD:")
print("="*80)

for threshold in thresholds:
    # Aplicar threshold customizado
    y_pred_custom = (y_pred_proba[:, 1] >= threshold).astype(int)
    
    # Calcular métricas
    acc = accuracy_score(y_test, y_pred_custom)
    cm = confusion_matrix(y_test, y_pred_custom)
    
    # Calcular precision e recall manualmente
    TP = cm[1, 1]  # True Positives
    FP = cm[0, 1]  # False Positives
    FN = cm[1, 0]  # False Negatives
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    
    print(f"\nThreshold = {threshold}")
    print(f"  Acurácia: {acc*100:.2f}%")
    print(f"  Precision: {precision*100:.2f}%")
    print(f"  Recall: {recall*100:.2f}%")
    print(f"  Matriz de Confusão: {cm.tolist()}")

### **Técnica 2: Regularização (L1 e L2)**

Regularização previne overfitting adicionando uma penalidade aos pesos grandes.

In [ ]:
# Comparando diferentes tipos de regularização

print("COMPARAÇÃO DE REGULARIZAÇÃO:")
print("="*80)

# Sem regularização (C muito alto)
model_no_reg = LogisticRegression(C=1e10, max_iter=1000, random_state=42)
model_no_reg.fit(X_train, y_train)
acc_no_reg = model_no_reg.score(X_test, y_test)

# Ridge (L2) - padrão
model_l2 = LogisticRegression(penalty='l2', C=1.0, max_iter=1000, random_state=42)
model_l2.fit(X_train, y_train)
acc_l2 = model_l2.score(X_test, y_test)

# Lasso (L1)
model_l1 = LogisticRegression(penalty='l1', solver='saga', C=1.0, max_iter=1000, random_state=42)
model_l1.fit(X_train, y_train)
acc_l1 = model_l1.score(X_test, y_test)

print(f"\nSem Regularização:")
print(f"  Acurácia: {acc_no_reg*100:.2f}%")
print(f"  Coeficientes: {model_no_reg.coef_[0]}")

print(f"\nRidge (L2):")
print(f"  Acurácia: {acc_l2*100:.2f}%")
print(f"  Coeficientes: {model_l2.coef_[0]}")

print(f"\nLasso (L1):")
print(f"  Acurácia: {acc_l1*100:.2f}%")
print(f"  Coeficientes: {model_l1.coef_[0]}")

print("\nObservação: L1 (Lasso) pode forçar coeficientes a zero,")
print("realizando seleção automática de features.")

<a id='vantagens-desvantagens'></a>
## **Vantagens e Desvantagens da Regressão Logística**

Agora que você domina a Regressão Logística, vamos resumir seus pontos fortes e fracos:

### **Vantagens**

1. **Rápida e eficiente:** Treina rapidamente mesmo em datasets grandes
2. **Fácil interpretação:** Coeficientes têm significado claro (odds ratios)
3. **Saída probabilística:** Fornece probabilidades, não apenas classes
4. **Não requer muitos dados:** Funciona bem com amostras relativamente pequenas
5. **Poucos hiperparâmetros:** Fácil de configurar e ajustar
6. **Naturalmente multiclasse:** Extensível via Softmax Regression
7. **Baseline excelente:** Ótimo ponto de partida para comparação

### **Desvantagens**

1. **Assume relação linear:** Não captura relações não-lineares entre features e target
2. **Sensível a multicolinearidade:** Não funciona bem quando features são muito correlacionadas
3. **Limitado em alta dimensionalidade:** Performance cai com muitas features irrelevantes
4. **Fronteira de decisão linear:** Não pode aprender fronteiras complexas
5. **Requer features bem escaladas:** Sensível a escala das variáveis
6. **Viés com classes desbalanceadas:** Tende a favorecer a classe majoritária
7. **Outliers influenciam:** Valores extremos podem afetar os coeficientes

### **Quando usar Regressão Logística?**

**Bom para:**
- Classificação binária simples
- Quando interpretabilidade é crucial (medicina, finanças)
- Baseline rápido para benchmark
- Datasets pequenos a médios
- Quando você precisa de probabilidades calibradas

**Evitar quando:**
- Relações complexas e não-lineares
- Features altamente correlacionadas
- Fronteiras de decisão complexas são necessárias
- Quando você pode sacrificar interpretabilidade por acurácia

### **Tipos de Regressão Logística**

#### **Regressão Logística Binária**

Funciona para problemas com apenas dois resultados possíveis. A variável dependente pode ter apenas dois valores, como sim/não ou 0/1. É o que usamos no exemplo do Titanic.

#### **Regressão Logística Multinomial**

Pode analisar problemas com vários resultados possíveis, desde que o número seja finito. Por exemplo, classificar emails em "spam", "promoção" ou "importante".

#### **Regressão Logística Ordinal**

Tipo especial de multinomial para problemas onde os números representam classificações ordenadas. Por exemplo, prever avaliações de 1 a 5 estrelas.

## **Recursos Adicionais**

Para aprofundar seu conhecimento em Regressão Logística:

- **Regularização:** [Logistic Regression and Regularization](https://medium.com/@rithpansanga/logistic-regression-and-regularization-avoiding-overfitting-and-improving-generalization-e9afdcddd09d)
- **Softmax Regression:** [Multinomial Logistic Regression](https://rasbt.github.io/mlxtend/user_guide/classifier/SoftmaxRegression/)
- **Fundamentos:** [Understanding Logistic Regression](https://www.geeksforgeeks.org/machine-learning/understanding-logistic-regression/)

---

<-- [**Anterior: Naive Bayes**](02_naive_bayes.ipynb) | [**Próximo: SVM**](04_svm.ipynb) -->